In [3]:
import sqlite3
import pandas as pd
import glob
import subprocess, os

In [4]:
'''
1. read file of cb_multios_execs
2. create dictionary k (original), v (patched)
3. cmd: bindiff orig.BinExport patched.BinExport
4. analysis (to get diff function names)
'''

'\n1. read file of cb_multios_execs\n2. create dictionary k (original), v (patched)\n3. cmd: bindiff orig.BinExport patched.BinExport\n4. analysis (to get diff function names)\n'

In [5]:
files = sorted([file for file in glob.glob('/home/user/Desktop/cbmultios_BinExports/*.BinExport')])

In [6]:
def make_pairs(files):
    originals = [file for file in files if "_patched" not in file]
    patched = [(file.replace('.BinExport', '_patched.BinExport')) for file in originals]
    pairs = dict(zip(originals, patched))
    #pairs = dict(zip(files[::2], files[1::2]))
    return pairs

In [7]:
def make_bindiff(original, patched):
    output_dir = '/home/user/Desktop/cbmultios_BinExports/bindiffs/'
    full_path = output_dir + original.split('/')[-1].replace('.BinExport','') + '_vs_' + patched.split('/')[-1].replace('.BinExport','') + '.BinDiff'
    cmd = ['/usr/bin/bindiff', original, patched, '--output_dir='+ output_dir]
    subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return full_path

In [8]:
def bindiff_to_csv(bindiff_diff):
    print('bindiff to csv: ', bindiff_diff)
    #bindiff_diff = bindiff_diff.replace('.BinExport','')
    db = sqlite3.connect(bindiff_diff)
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    try:
        fun_table = tables[4]
        #table = pd.read_sql_query("SELECT * from %s" % fun_table, db)
        table = pd.read_sql_query("SELECT * from %s" % fun_table, db)

        output_dir = '/home/user/Desktop/cbmultios_BinExports/bindiffs/funtables/'
        name = output_dir + bindiff_diff.split('/')[-1] + '_funtable.csv'
        table.to_csv(name, index_label='index')
        cursor.close()
        db.close()
        return name

    except:
        return 'error'

In [37]:
def get_fcn(name_sample_addr):
    try:
        index = fun_df.index[fun_df['sample_name_addr'] == name_sample_addr].tolist()[0]
        fun_name = fun_df.iloc[index]['fun_name']
    except:
        fun_name='not_found'
    return fun_name


In [38]:
def coorelate_fun_names(table_path):
    bindiff_df = pd.read_csv(table_path, index_col=[0])
    name = table_path.split('/')[-1].replace('_funtable.csv','').replace('.BinDiff','').split('_vs_')
    
    #index,id,address1,address2,similarity,confidence,
    #flags,algorithm,evaluate,commentsported,basicblocks,edges,instructions
    bindiff_df['original_sample'] = name[0] #get original sample name
    bindiff_df['patched_sample'] = name[1] #get patched sample name

    bindiff_df['original_sample_addr'] = bindiff_df['original_sample'] + '_' + bindiff_df['address1'].apply(str)
    bindiff_df['patched_sample_addr'] = bindiff_df['patched_sample'] + '_' + bindiff_df['address2'].apply(str)

    bindiff_df['orig_fun_name'] = bindiff_df['original_sample_addr'].apply(lambda x: get_fcn(x))
    bindiff_df['patched_fun_name'] = bindiff_df['patched_sample_addr'].apply(lambda x: get_fcn(x))

    csv_name = base_dir + table_path.split('/')[-1].replace('.BinDiff','') + '_finaldf.csv'
    bindiff_df.to_csv(csv_name)
    return bindiff_df


In [33]:
base_dir = '/home/user/Desktop/cbmultios_BinExports/ghidra_function_data/'
fun_df = pd.read_csv(base_dir+'sample_functions_addresses.csv')
fun_df.columns = ['path', 'fun_name', 'addr']
fun_df['name'] = fun_df['path'].apply(lambda x: x.split('/')[-1])
fun_df['sample_name_addr'] = fun_df['name'] + '_' + fun_df['addr'].apply(str)

In [34]:
fun_df

,path,fun_name,addr,name,sample_name_addr
0,/home/user/Desktop/cb_multios_binaries_origina...,FUN_0804a030,134520880,3D_Image_Toolkit,3D_Image_Toolkit_134520880
1,/home/user/Desktop/cb_multios_binaries_origina...,cgc_pow,134520896,3D_Image_Toolkit,3D_Image_Toolkit_134520896
2,/home/user/Desktop/cb_multios_binaries_origina...,cgc__terminate,134520912,3D_Image_Toolkit,3D_Image_Toolkit_134520912
3,/home/user/Desktop/cb_multios_binaries_origina...,cgc_transmit,134520928,3D_Image_Toolkit,3D_Image_Toolkit_134520928
4,/home/user/Desktop/cb_multios_binaries_origina...,cgc_cos,134520944,3D_Image_Toolkit,3D_Image_Toolkit_134520944
...,...,...,...,...,...
2382,/home/user/Desktop/cb_multios_binaries_origina...,cgc_log,134541344,Audio_Visualizer,Audio_Visualizer_134541344
2383,/home/user/Desktop/cb_multios_binaries_origina...,cgc_deallocate,134541348,Audio_Visualizer,Audio_Visualizer_134541348
2384,/home/user/Desktop/cb_multios_binaries_origina...,cgc_sin,134541352,Audio_Visualizer,Audio_Visualizer_134541352
2385,/home/user/Desktop/cb_multios_binaries_origina...,cgc_sqrt,134541356,Audio_Visualizer,Audio_Visualizer_134541356


In [12]:
pairs = make_pairs(files)

In [14]:
bindiffs = [make_bindiff(k, v) for k, v in pairs.items()]

In [ ]:
bindiff_fcn_paths = [bindiff_to_csv(bindiff) for bindiff in bindiffs]

In [ ]:
coorelate_fun_names(bindiff_fcn_paths[0])

In [ ]:
final_dfs = [coorelate_fun_names(bindiff_fcn_path) for bindiff_fcn_path in bindiff_fcn_paths if bindiff_fcn_path != 'error']

In [ ]:
final_combined = pd.concat(final_dfs, axis=0)

In [ ]:
final_combined.to_csv('/home/user/Desktop/cbmultios_BinExports/resources/final_combined.csv')